In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd


path = "/content/drive/My Drive/fire_predict/base_data/"
df_train = pd.read_csv(path + "PJT002_train.csv")
df_val = pd.read_csv(path + "PJT002_validation.csv")
df_test = pd.read_csv(path + "PJT002_test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


def f_data(df_train, df_val, df_test):

    X_train = df_train.drop(['fr_yn'], 1)
    y_train = df_train['fr_yn']
    X_val = df_val.drop(['fr_yn'], 1)
    y_val = df_val['fr_yn']
    X_test = df_test

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(df_train)]
    X_val = df_all[len(df_train):-len(df_test)]
    X_test = df_all[-len(df_test):]

    X_train = X_train.fillna(-1)
    X_val = X_val.fillna(-1)
    X_test = X_test.fillna(-1)

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

# lable -> binary
binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

#df ->df_2
df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

#drop id
df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

#drop dt_of_fr
df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

#dt_of_athrztn -> year 
def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])

df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

# hm_cnt -> categorical
def population(x):
    if x < 1000:
        return "1k"
    elif x < 10000:
        return "10k"
    elif x < 100000:
        return "100k"
    elif x < 1000000:
        return "1000k"

df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

#tmprtr c -> k
df_2.tmprtr = df_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

#gas_engry_us_201507 x < 0 -> remove
df_2.gas_engry_us_201507 = df_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

# drop gas, elec
df_2 = df_2.drop(df_2.iloc[:,16:136].columns, axis=1)
df_2

# X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
# f1_rfc(X_train, y_train, X_val, y_val)

NameError: ignored

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
import numpy as np


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [0]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=  41.6s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   41.6s remaining:    0.0s


[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=  41.8s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=  37.2s
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total= 1.9min
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total= 1.9min
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=2000, min_sa

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 634.5min finished


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 200}

In [0]:
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [0]:
rf = RandomForestClassifier(bootstrap=True, max_depth=80, max_features="auto", min_samples_leaf=4, min_samples_split=5, n_estimators=200)

In [0]:


def evaluate(model, X_val, y_val):
    predictions = model.predict(X_val)
    errors = abs(predictions - y_val)
    mape = 100 * np.mean(errors / y_val)
    accuracy = 100 - mape
    f1 = f1_score(y_val, predictions)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('f1 = {}'.format(f1))
    
    return f1

base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_val, y_val)

rf.fit(X_train, y_train)
best_random = rf
random_accuracy = evaluate(best_random, X_val, y_val)

Model Performance
Average Error: 0.2044 degrees.
Accuracy = -inf%.
f1 = 0.4409199048374306
Model Performance
Average Error: 0.1735 degrees.
Accuracy = -inf%.
f1 = 0.5006257822277848


In [0]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, verbose = 2)

In [0]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_val, y_val)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   2.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   2.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   1.9s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200, total=   4.1s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200, total=   4.0s
[CV] bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, max_features=2, min_samp

[Parallel(n_jobs=1)]: Done 864 out of 864 | elapsed: 124.9min finished


Model Performance
Average Error: 0.1631 degrees.
Accuracy = -inf%.
f1 = 0.4477172312223858
